## Airbnb Market Analysis and Price Prediction Project

This project explores New York City Airbnb data from Inside Airbnb, with a focus on uncovering market patterns and predicting listing prices using machine learning. We analyze trends in reviews, availability in listing characteristics to help inform pricing strategy and highlight high-performing listings

In [38]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [39]:
#loading dataphrames
calendar=pd.read_csv('/Users/zoewalp/Desktop/DAProjects/airbnb_nyc/nyc_calendar.csv')
reviews=pd.read_csv('/Users/zoewalp/Desktop/DAProjects/airbnb_nyc/nyc_reviews.csv')
listings=pd.read_csv('/Users/zoewalp/Desktop/DAProjects/airbnb_nyc/nyc_listings.csv')

### Examining Calendar Dataframe

In [40]:
#previewing data in calendar dataframe
calendar.head()
calendar.info()
calendar.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   listing_id      999999 non-null  int64  
 1   date            999999 non-null  object 
 2   available       999999 non-null  object 
 3   price           999999 non-null  object 
 4   adjusted_price  0 non-null       float64
 5   minimum_nights  999636 non-null  float64
 6   maximum_nights  999636 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 53.4+ MB


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
count,9.999990e+05,999999,999999,999999,0.0,999636.000000,999636.000000
unique,NaN,366,2,291,NaN,NaN,NaN
top,NaN,2025-05-02,f,$150.00,NaN,NaN,NaN
freq,NaN,2741,708182,55845,NaN,NaN,NaN
mean,2.511877e+06,NaN,NaN,NaN,NaN,33.959834,706.838358
std,1.746287e+06,NaN,NaN,NaN,NaN,48.877865,550.669339
min,2.539000e+03,NaN,NaN,NaN,NaN,1.000000,3.000000
25%,7.980080e+05,NaN,NaN,NaN,NaN,30.000000,95.000000
50%,2.298731e+06,NaN,NaN,NaN,NaN,30.000000,1125.000000
75%,4.121173e+06,NaN,NaN,NaN,NaN,30.000000,1125.000000


In [41]:
#check for missing values

calendar.isnull().sum()

listing_id             0
date                   0
available              0
price                  0
adjusted_price    999999
minimum_nights       363
maximum_nights       363
dtype: int64

In [53]:
#note that there are 363 'minimum_nights' and 363 'maximum_nights'
#dropping adjusted price column since nearly all entries are missing
if 'adjusted_price' in calendar.columns:
    calendar_cleaned = calendar.drop(columns=['adjusted_price'])
else:
    calendar_cleaned = calendar.copy()  # just copy if column not present


### Examining Reviews Dataframe

In [43]:
#previewing data in reviews dataframe
reviews.head()
reviews.info()
reviews.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971379 entries, 0 to 971378
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     971379 non-null  int64 
 1   id             971379 non-null  int64 
 2   date           971379 non-null  object
 3   reviewer_id    971379 non-null  int64 
 4   reviewer_name  971375 non-null  object
 5   comments       971127 non-null  object
dtypes: int64(3), object(3)
memory usage: 44.5+ MB


,listing_id,id,date,reviewer_id,reviewer_name,comments
count,9.713790e+05,9.713790e+05,971379,9.713790e+05,971375,971127
unique,NaN,NaN,5431,NaN,120946,933198
top,NaN,NaN,2023-09-04,NaN,David,.
freq,NaN,NaN,1234,NaN,6887,1181
mean,1.828369e+17,5.239370e+17,NaN,1.657245e+08,NaN,NaN
std,3.554860e+17,4.967769e+17,NaN,1.616621e+08,NaN,NaN
min,2.539000e+03,3.149000e+03,NaN,2.900000e+01,NaN,NaN
25%,1.028055e+07,4.169636e+08,NaN,3.227757e+07,NaN,NaN
50%,2.863504e+07,5.724516e+17,NaN,1.090590e+08,NaN,NaN
75%,5.266669e+07,9.471487e+17,NaN,2.581540e+08,NaN,NaN


In [44]:
#convert date to datetime
reviews['date'] = pd.to_datetime(reviews['date'])

In [45]:
#looking at duplicates
reviews.duplicated().sum()
reviews.drop_duplicates(inplace=True)

In [46]:
reviews_cleaned = reviews.copy()

### Examining Listings Dataframe

In [47]:
#previewing data in listings dataframe
listings.head()
listings.info()
listings.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37018 entries, 0 to 37017
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            37018 non-null  int64  
 1   listing_url                                   37018 non-null  object 
 2   scrape_id                                     37018 non-null  int64  
 3   last_scraped                                  37018 non-null  object 
 4   source                                        37018 non-null  object 
 5   name                                          37016 non-null  object 
 6   description                                   36010 non-null  object 
 7   neighborhood_overview                         19575 non-null  object 
 8   picture_url                                   37017 non-null  object 
 9   host_id                                       37018 non-null 

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,3.701800e+04,37018,3.701800e+04,37018,37018,37016,36010,19575,37017,3.701800e+04,...,25352.000000,25341.000000,25342.000000,5396,37018,37018.000000,37018.000000,37018.000000,37018.000000,25358.000000
unique,NaN,37018,NaN,3,2,35355,30614,14677,36238,NaN,...,NaN,NaN,NaN,2006,2,NaN,NaN,NaN,NaN,NaN
top,NaN,https://www.airbnb.com/rooms/2539,NaN,2025-05-02,city scrape,Water View King Bed Hotel Room,Keep it simple at this peaceful and centrally-...,This furnished apartment is located in the Fin...,https://a0.muscache.com/pictures/6998e77e-4564...,NaN,...,NaN,NaN,NaN,Exempt,f,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,27161,22056,30,93,101,35,NaN,...,NaN,NaN,NaN,2844,29409,NaN,NaN,NaN,NaN,NaN
mean,4.372920e+17,NaN,2.025050e+13,NaN,NaN,NaN,NaN,NaN,NaN,1.714205e+08,...,4.826887,4.744965,4.644073,NaN,NaN,72.739802,51.326436,19.406262,0.029094,0.819498
std,5.144149e+17,NaN,1.528536e+01,NaN,NaN,NaN,NaN,NaN,NaN,1.888658e+08,...,0.408087,0.390465,0.486165,NaN,NaN,235.286250,221.236309,89.511742,0.406795,1.851575
min,2.539000e+03,NaN,2.025050e+13,NaN,NaN,NaN,NaN,NaN,NaN,1.678000e+03,...,0.000000,0.000000,0.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.010000
25%,2.140000e+07,NaN,2.025050e+13,NaN,NaN,NaN,NaN,NaN,NaN,1.768211e+07,...,4.820000,4.660000,4.540000,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.080000
50%,5.037505e+07,NaN,2.025050e+13,NaN,NaN,NaN,NaN,NaN,NaN,8.655362e+07,...,4.960000,4.850000,4.765000,NaN,NaN,2.000000,1.000000,0.000000,0.000000,0.260000
75%,9.261013e+17,NaN,2.025050e+13,NaN,NaN,NaN,NaN,NaN,NaN,3.036643e+08,...,5.000000,5.000000,4.950000,NaN,NaN,8.000000,2.000000,2.000000,0.000000,0.940000


In [48]:
#checking for missing values
listings.isnull().sum()

id                                                  0
listing_url                                         0
scrape_id                                           0
last_scraped                                        0
source                                              0
                                                ...  
calculated_host_listings_count                      0
calculated_host_listings_count_entire_homes         0
calculated_host_listings_count_private_rooms        0
calculated_host_listings_count_shared_rooms         0
reviews_per_month                               11660
Length: 79, dtype: int64

In [49]:
#seeing if missing values >50% of columns

missing = listings.isnull().sum()
missing_percent = (missing / len(listings)) * 100
missing_percent[missing_percent>50].sort_values(ascending=False)

calendar_updated    100.000000
license              85.423308
dtype: float64

In [50]:
# Step 1: Calculate missing values and percentage
missing = listings.isnull().sum()
missing_percent = (missing / len(listings)) * 100

# Step 2: Identify columns to drop (>50% missing)
cols_to_drop = missing_percent[missing_percent > 50].index

# Step 3: Drop those columns and create a cleaned copy
listings_cleaned = listings.drop(columns=cols_to_drop)
listings_cleaned = listings_cleaned.copy()

# Step 4: Optional — confirm the shape
print(f"Original shape: {listings.shape}")
print(f"Cleaned shape: {listings_cleaned.shape}")


Original shape: (37018, 79)
Cleaned shape: (37018, 77)


In [56]:
reviews_cleaned.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2539,55688172,2015-12-04,25160947,Peter,Great host
1,2539,97474898,2016-08-27,91513326,Liz,Nice room for the price. Great neighborhood. J...
2,2539,105340344,2016-10-01,90022459,Евгений,Very nice apt. New remodeled.
3,2539,133131670,2017-02-20,116165195,George,Great place to stay for a while. John is a gre...
4,2539,138349776,2017-03-19,118432644,Carlos,.
